## First attempt

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Load the cleaned data
df = pd.read_excel('../../data/clean_data/cleaned_UNAIDS.xlsx')

# Define UN regions
un_regions = ["UNAAP", "UNACAR", "UNAESA", "UNAEECA", "UNALA", "UNAMENA", "UNAWCA", "UNAWCENA"]

# Function to create the Sankey diagram for a specific year
def create_sankey_diagram(year, include_regions=False):
    # Filter data for the specified year
    year_data = df[df['Year'] == year].copy()
    
    # Replace NaN values with 0 for calculation purposes
    year_data['AIDS-related deaths among adults and children'] = year_data['AIDS-related deaths among adults and children'].fillna(0)
    year_data['Estimated adults and children living with HIV'] = year_data['Estimated adults and children living with HIV'].fillna(0)
    
    # Calculate non-AIDS HIV cases (people living with HIV who don't die from AIDS)
    year_data['Living with HIV (non-AIDS deaths)'] = year_data['Estimated adults and children living with HIV'] - year_data['AIDS-related deaths among adults and children']
    year_data['Living with HIV (non-AIDS deaths)'] = year_data['Living with HIV (non-AIDS deaths)'].clip(lower=0)  # Ensure no negative numbers
    
    # Prepare basic nodes and links
    nodes = ['People Living with HIV', 'AIDS-related Deaths', 'Living with HIV (non-AIDS)']
    node_colors = ['blue', 'red', 'green']
    
    # Global totals
    total_hiv = year_data['Estimated adults and children living with HIV'].sum()
    total_aids_deaths = year_data['AIDS-related deaths among adults and children'].sum()
    total_non_aids = total_hiv - total_aids_deaths
    
    if not include_regions:
        # Simple flow: HIV → AIDS deaths or non-AIDS
        sources = [0, 0]
        targets = [1, 2]
        values = [total_aids_deaths, total_non_aids]
        colors = ['rgba(255, 0, 0, 0.4)', 'rgba(0, 255, 0, 0.4)']
        
        title = f'HIV and AIDS Flow Diagram ({year})'
    
    else:
        # Add region nodes to the basic nodes
        region_indices = {}
        for i, region in enumerate(un_regions):
            region_filtered = year_data[year_data['Country Code'] == region]
            if len(region_filtered) > 0:
                nodes.append(region)
                node_colors.append(px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)])
                region_indices[region] = len(nodes) - 1
        
        # Create sources, targets, and values arrays
        sources = []
        targets = []
        values = []
        colors = []
        
        # First, add connections from HIV total to regions
        for region, region_idx in region_indices.items():
            region_data = year_data[year_data['Country Code'] == region]
            region_hiv = region_data['Estimated adults and children living with HIV'].sum()
            
            if region_hiv > 0:
                sources.append(0)  # From HIV total
                targets.append(region_idx)  # To region
                values.append(region_hiv)
                colors.append('rgba(0, 0, 255, 0.3)')  # Blue for HIV
        
        # Then add connections from regions to outcomes
        for region, region_idx in region_indices.items():
            region_data = year_data[year_data['Country Code'] == region]
            region_aids_deaths = region_data['AIDS-related deaths among adults and children'].sum()
            region_non_aids = region_data['Living with HIV (non-AIDS deaths)'].sum()
            
            if region_aids_deaths > 0:
                sources.append(region_idx)  # From region
                targets.append(1)  # To AIDS deaths
                values.append(region_aids_deaths)
                colors.append('rgba(255, 0, 0, 0.4)')  # Red for AIDS deaths
            
            if region_non_aids > 0:
                sources.append(region_idx)  # From region
                targets.append(2)  # To non-AIDS
                values.append(region_non_aids)
                colors.append('rgba(0, 255, 0, 0.4)')  # Green for non-AIDS
        
        title = f'HIV and AIDS Flow Diagram by Region ({year})'

    # Create the Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "#121212", width = 0.5),
            label = nodes,
            color = node_colors
        ),
        link = dict(
            source = sources,
            target = targets,
            value = values,
            color = colors
        )
    )])
    
    fig.update_layout(title_text=title, font_size=12)
    
    return fig

# Create figures for 1990 and 2022 (or the latest year in your dataset if 2022 is not available)
latest_year = df['Year'].max()
target_year = 2022 if 2022 in df['Year'].unique() else latest_year

# Create basic diagrams
fig1990_basic = create_sankey_diagram(1990, include_regions=False)
fig_latest_basic = create_sankey_diagram(target_year, include_regions=False)

# Create diagrams with regional breakdown
fig1990_regions = create_sankey_diagram(1990, include_regions=True)
fig_latest_regions = create_sankey_diagram(target_year, include_regions=True)

# Create a subplot with buttons to switch between views
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "sankey"}, {"type": "sankey"}]])

# Create combined figure with buttons for interactivity
fig = go.Figure()

# Add the basic diagrams as the default view
fig.add_trace(fig1990_basic.data[0])
fig.add_trace(fig_latest_basic.data[0])

# Update layout
fig.update_layout(
    title="HIV and AIDS Flow Diagrams (1990 vs Latest Year)",
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(
                    label="Basic View",
                    method="update",
                    args=[{"visible": [True, True, False, False]},
                          {"title": "HIV and AIDS Flow Diagrams - Basic View"}]
                ),
                dict(
                    label="Regional Breakdown",
                    method="update",
                    args=[{"visible": [False, False, True, True]},
                          {"title": "HIV and AIDS Flow Diagrams - Regional Breakdown"}]
                )
            ]),
        )
    ]
)

# Add regional diagrams (initially hidden)
for trace in fig1990_regions.data:
    fig.add_trace(trace)
    fig.data[-1].visible = False

for trace in fig_latest_regions.data:
    fig.add_trace(trace)
    fig.data[-1].visible = False

# Add annotations
fig.add_annotation(
    x=0.25, y=1.05,
    xref="paper", yref="paper",
    text=f"1990",
    showarrow=False,
    font=dict(size=16)
)

fig.add_annotation(
    x=0.75, y=1.05,
    xref="paper", yref="paper",
    text=f"{target_year}",
    showarrow=False,
    font=dict(size=16)
)

# Add explanatory annotation
fig.add_annotation(
    x=0.5, y=-0.15,
    xref="paper", yref="paper",
    text="This visualization shows that HIV and AIDS are not the same thing.<br>Many people live with HIV without developing AIDS,<br>especially with modern treatments.",
    showarrow=False,
    font=dict(size=14)
)

# Show the figure
fig.show()

# Save the figure as an HTML file for interactive viewing
fig.write_html('../../interactive_viz_outputs/hiv_aids_flow_diagram.html')

print("Visualization created and saved as 'hiv_aids_flow_diagram.html'")

Visualization created and saved as 'hiv_aids_flow_diagram.html'


## Final attempt with styling

In [2]:
import pandas as pd
import json
import os

df = pd.read_excel('../../data/clean_data/cleaned_UNAIDS.xlsx')
file_path = '../../data/clean_data/cleaned_UNAIDS.xlsx'
available_years = sorted(df['Year'].unique())
start_year = min(available_years)
end_year = max(available_years)


all_years_data = []

# Find total count of HIV cases in 1990 (for starting point text)
hiv_1990_data = df[df['Year'] == 1990].copy()
total_hiv_1990 = int(hiv_1990_data['Estimated adults and children living with HIV'].sum())
formatted_hiv_1990 = "{:,}".format(total_hiv_1990)

for year in available_years:
    # specify year
    year_data = df[df['Year'] == year].copy()
    
    # data cleaning
    year_data['AIDS-related deaths among adults and children'] = year_data['AIDS-related deaths among adults and children'].fillna(0)
    year_data['Estimated adults and children living with HIV'] = year_data['Estimated adults and children living with HIV'].fillna(0)
    
    # non-AIDS HIV cases
    year_data['Living with HIV (non-AIDS deaths)'] = year_data['Estimated adults and children living with HIV'] - year_data['AIDS-related deaths among adults and children']
    year_data['Living with HIV (non-AIDS deaths)'] = year_data['Living with HIV (non-AIDS deaths)'].clip(lower=0)
    
    # global totals
    total_hiv = year_data['Estimated adults and children living with HIV'].sum()
    total_aids_deaths = year_data['AIDS-related deaths among adults and children'].sum()
    total_non_aids = total_hiv - total_aids_deaths
    
    # percentages --> % of people ending up living with HIV or dying from AIDS related causes
    death_percentage = (total_aids_deaths / total_hiv * 100) if total_hiv > 0 else 0
    survival_percentage = (total_non_aids / total_hiv * 100) if total_hiv > 0 else 0
    
    all_years_data.append({
        'year': int(year),
        'total_hiv': float(total_hiv),
        'aids_deaths': float(total_aids_deaths),
        'non_aids': float(total_non_aids),
        'death_percentage': float(death_percentage),
        'survival_percentage': float(survival_percentage)
    })

# Convert data to json
json_data = json.dumps(all_years_data)

# html file w/ embedded data & d3.js
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HIV and AIDS Flow Diagram ({start_year}-{end_year})</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://unpkg.com/d3-sankey@0.12.3/dist/d3-sankey.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #121212;
            color: white;
        }}
        
        .container {{
            max-width: 1000px;
            margin: 0 auto;
        }}
        
        h1 {{
            text-align: center;
            margin-bottom: 20px;
        }}
        
        .controls {{
            display: flex;
            justify-content: flex-start;
            margin-bottom: 20px;
            gap: 10px;
        }}
        
        button {{
            padding: 10px 20px;
            background-color: #333;
            color: white;
            border: 1px solid #666;
            border-radius: 4px;
            cursor: pointer;
        }}
        
        button:hover {{
            background-color: #444;
        }}
        
        .year-display {{
            text-align: center;
            font-size: 24px;
            margin-bottom: 20px;
        }}
        
        #sankey-container {{
            width: 100%;
            height: 450px; 
            position: relative;
            margin: 0 auto;
        }}
        
        #year-slider {{
            width: 100%;
            margin: 20px 0;
        }}
        
        .node rect {{
            cursor: move;
            fill-opacity: 0.9;
            shape-rendering: crispEdges;
        }}
        
        .node text {{
            pointer-events: none;
            text-shadow: 0 1px 0 #000;
        }}
        
        .link {{
            fill: none;
            stroke-opacity: 0.5;
        }}
        
        .tooltip {{
            position: absolute;
            background-color: white;
            color: #121212;
            padding: 10px;
            border-radius: 5px;
            pointer-events: none;
            box-shadow: 0 0 10px rgba(0,0,0,0.5);
        }}
        
        .explanation {{
            text-align: center;
            margin-top: 30px;
            margin-bottom: 20px;
            font-style: italic;
        }}
        
        /* STYLING FOR STATS BOXES --> MODIFY POSITIONS HERE */
        .stats-box {{
            position: absolute;
            padding: 10px;
            border-radius: 4px;
            pointer-events: none;
        }}
        
        .hiv-stats {{
            top: 35%;  /* ADJUST TO MOVE box up/down */
            right: 10px;  /* ADJUST TO MOVE box left/right */
            background-color: white;
            color: #fc9097;
            border: 2px solid #fc9097;
        }}
        
        .aids-stats {{
            bottom: 10%;  /* ADJUST TO MOVE box up/down */
            right: 10px;  /* ADJUST TO MOVE box left/right */
            background-color: white;
            color: red;
            border: 2px solid red;
        }}
        
        /* STYLING FOR STARTING POINT BOX */
        .starting-note {{
            position: absolute;
            top: 25%;  /* ADJUST TO MOVE box up/down */
            left: 80px;  /* ADJUST TO MOVE box left/right */
            background-color: #121212;
            color: #fc656f;  
            border: 2px solid #fc656f;
            padding: 10px;
            border-radius: 4px;
            pointer-events: none;
        }}
        
    </style>
</head>
<body>
    <div class="container">
        <h1>HIV and AIDS Flow Diagram ({start_year}-{end_year})</h1>
        
        <div class="controls">
            <button id="play-button">Play</button>
            <button id="pause-button">Pause</button>
        </div>
        
        <div class="year-display" id="year-display">Year: {start_year}</div>
        
        <div id="sankey-container">
            <div class="starting-note">All {formatted_hiv_1990} HIV cases in 1990</div>
            <div class="stats-box hiv-stats" id="hiv-stats"></div>
            <div class="stats-box aids-stats" id="aids-stats"></div>
        </div>
        
        <input type="range" id="year-slider" min="0" max="{len(available_years)-1}" value="0" step="1">
        
        <div class="explanation">
            This visualization shows how the proportion of AIDS-related deaths has decreased over time.<br>
            Modern treatments have allowed more people to live with HIV without developing AIDS.
        </div>
    </div>
    
    <script>
        const allYearsData = {json_data};
        const availableYears = allYearsData.map(d => d.year);

        let currentYearIndex = 0;
        let animationInterval;
        let isPlaying = false;
        
        // get DOM elements
        const sankeyContainer = document.getElementById('sankey-container');
        const yearDisplay = document.getElementById('year-display');
        const yearSlider = document.getElementById('year-slider');
        const playButton = document.getElementById('play-button');
        const pauseButton = document.getElementById('pause-button');
        const hivStats = document.getElementById('hiv-stats');
        const aidsStats = document.getElementById('aids-stats');
        
        // set up sankey
        const margin = {{top: 10, right: 100, bottom: 10, left: 100}};
        const width = sankeyContainer.clientWidth - margin.left - margin.right;
        const height = sankeyContainer.clientHeight - margin.top - margin.bottom;
        
        const svg = d3.select('#sankey-container')
            .append('svg')
            .attr('width', width + margin.left + margin.right)
            .attr('height', height + margin.top + margin.bottom)
            .append('g')
            .attr('transform', `translate(${{margin.left}},${{margin.top}})`);

        updateSankey(allYearsData[0]);

        playButton.addEventListener('click', startAnimation);
        pauseButton.addEventListener('click', pauseAnimation);
        yearSlider.addEventListener('input', handleSliderChange);
        
        // RESIZE WINDOW !!!!!!
        window.addEventListener('resize', () => {{
            // Redraw on window resize
            updateSankey(allYearsData[currentYearIndex]);
        }});
        
        // funcs
        function updateSankey(yearData) {{
            // clear previous 
            svg.selectAll('*').remove();
            
            // update year display
            yearDisplay.textContent = `Year: ${{yearData.year}}`;
            
            // update stats
            hivStats.textContent = `Living with HIV: ${{yearData.survival_percentage.toFixed(1)}}%`;
            aidsStats.textContent = `AIDS-related Deaths: ${{yearData.death_percentage.toFixed(1)}}%`;

            const totalFlow = yearData.aids_deaths + yearData.non_aids;
            const totalHeight = height * 0.8; // Use 80% of available height

            const hivNodeHeight = totalHeight;
            const nonAidsNodeHeight = (yearData.non_aids / totalFlow) * totalHeight;
            const aidsNodeHeight = (yearData.aids_deaths / totalFlow) * totalHeight;

            const nodes = [
                {{ 
                    id: 0,
                    x0: 0,
                    x1: 20,
                    y0: (height - hivNodeHeight) / 2,
                    y1: (height - hivNodeHeight) / 2 + hivNodeHeight
                }},
                {{ 
                    id: 1,
                    x0: width - 20,
                    x1: width,
                    y0: height - aidsNodeHeight,
                    y1: height
                }},
                {{ 
                    id: 2,
                    x0: width - 20,
                    x1: width,
                    y0: 0,
                    y1: nonAidsNodeHeight
                }}
            ];
            
            // Create links with proper values
            const links = [
                {{ 
                    source: nodes[0], 
                    target: nodes[1], 
                    value: yearData.aids_deaths,
                    width: aidsNodeHeight,
                    color: 'rgba(255, 0, 0, 0.6)'
                }},
                {{ 
                    source: nodes[0], 
                    target: nodes[2], 
                    value: yearData.non_aids,
                    width: nonAidsNodeHeight,
                    color: 'rgba(252, 144, 151, 0.6)'
                }}
            ];
            
            // color scale
            const nodeColors = {{'0': '#fc656f', '1': 'red', '2': '#fc9097'}};
            
            // nodes
            const nodes_g = svg.append('g')
                .attr('class', 'nodes')
                .selectAll('g')
                .data(nodes)
                .enter()
                .append('g');
            
            nodes_g.append('rect')
                .attr('x', d => d.x0)
                .attr('y', d => d.y0)
                .attr('height', d => d.y1 - d.y0)
                .attr('width', d => d.x1 - d.x0)
                .style('fill', d => nodeColors[d.id])
                .style('stroke', '#fff');

            svg.append('g')
                .attr('class', 'links')
                .selectAll('path')
                .data(links)
                .enter()
                .append('path')
                .attr('d', d => {{
                    const sourceX = d.source.x1;
                    const sourceY = d.source.y0 + (d.source.y1 - d.source.y0) / 2;
                    const targetX = d.target.x0;
                    const targetY = d.target.y0 + (d.target.y1 - d.target.y0) / 2;

                    const controlX1 = sourceX + (targetX - sourceX) / 3;
                    const controlX2 = sourceX + 2 * (targetX - sourceX) / 3;
                    
                    // adjust for flow target(AIDS deaths = bottom, HIV = top)
                    const controlY1 = sourceY;
                    const controlY2 = targetY;

                    return `M${{sourceX}},${{sourceY}} C${{controlX1}},${{controlY1}} ${{controlX2}},${{controlY2}} ${{targetX}},${{targetY}}`;
                }})
                .attr('stroke', d => d.color)
                .attr('stroke-width', d => Math.max(1, d.width))
                .style('fill', 'none');
        }}
        
        function startAnimation() {{
            if (isPlaying) return;
            
            isPlaying = true;
            animationInterval = setInterval(() => {{
                currentYearIndex = (currentYearIndex + 1) % availableYears.length;
                yearSlider.value = currentYearIndex;
                updateSankey(allYearsData[currentYearIndex]);
                
                if (currentYearIndex === availableYears.length - 1) {{
                    // Reached the end
                    pauseAnimation();
                }}
            }}, 500); // 500ms per frame
        }}
        
        function pauseAnimation() {{
            clearInterval(animationInterval);
            isPlaying = false;
        }}
        
        function handleSliderChange() {{
            pauseAnimation();
            
            // update year
            currentYearIndex = parseInt(yearSlider.value);
            updateSankey(allYearsData[currentYearIndex]);
        }}
    </script>
</body>
</html>
"""

os.makedirs('../../interactive_viz_outputs', exist_ok=True)

# save to interactive_viz_outputs
output_path = '../../interactive_viz_outputs/final_styled_hiv_aids_flow_diagram.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

In [18]:
import pandas as pd
import json
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the cleaned data
df = pd.read_excel('../../data/clean_data/cleaned_UNAIDS.xlsx')

# Define UN regions
un_regions = ["UNAAP", "UNACAR", "UNAESA", "UNAEECA", "UNALA", "UNAMENA", "UNAWCA", "UNAWCENA"]

# Get available years
available_years = sorted(df['Year'].unique())
start_year = min(available_years)
end_year = max(available_years)

all_years_data_regions_flow = {}

for year in available_years:
    year_data = df[df['Year'] == year].copy()
    year_regions_data = {}
    total_hiv_global = year_data['Estimated adults and children living with HIV'].sum()

    for region in un_regions:
        region_data = year_data[year_data['Country Code'] == region].copy()
        region_hiv = region_data['Estimated adults and children living with HIV'].sum()
        region_aids_deaths = region_data['AIDS-related deaths among adults and children'].fillna(0).sum()
        region_non_aids = region_hiv - region_aids_deaths
        region_non_aids = max(0, region_non_aids)  # Ensure no negative values

        year_regions_data[region] = {
            'hiv': float(region_hiv),
            'aids_deaths': float(region_aids_deaths),
            'non_aids': float(region_non_aids),
        }
    all_years_data_regions_flow[int(year)] = {'global_hiv': float(total_hiv_global), 'regions': year_regions_data}

# Convert data to JSON
json_data_regions_flow = json.dumps(all_years_data_regions_flow)

# HTML file with embedded data and D3.js
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HIV and AIDS Flow by UN Region ({start_year}-{end_year})</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://unpkg.com/d3-sankey@0.12.3/dist/d3-sankey.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #121212;
            color: white;
        }}

        .container {{
            max-width: 1200px;
            margin: 0 auto;
        }}

        h1 {{
            text-align: center;
            margin-bottom: 20px;
        }}

        .controls {{
            display: flex;
            justify-content: flex-start;
            margin-bottom: 20px;
            gap: 10px;
        }}

        button {{
            padding: 10px 20px;
            background-color: #333;
            color: white;
            border: 1px solid #666;
            border-radius: 4px;
            cursor: pointer;
        }}

        button:hover {{
            background-color: #444;
        }}

        .year-display {{
            text-align: center;
            font-size: 24px;
            margin-bottom: 20px;
        }}

        #sankey-container {{
            width: 100%;
            height: 700px; /* Increased height to accommodate more nodes */
            position: relative;
            margin: 0 auto;
        }}

        #year-slider {{
            width: 100%;
            margin: 20px 0;
        }}

        .node rect {{
            cursor: move;
            fill-opacity: 0.9;
            shape-rendering: crispEdges;
        }}

        .node text {{
            pointer-events: none;
            text-shadow: 0 1px 0 #000;
        }}

        .link {{
            fill: none;
            stroke-opacity: 0.5;
        }}

        .tooltip {{
            position: absolute;
            background-color: white;
            color: #121212;
            padding: 10px;
            border-radius: 5px;
            pointer-events: none;
            box-shadow: 0 0 10px rgba(0,0,0,0.5);
        }}

        .explanation {{
            text-align: center;
            margin-top: 30px;
            margin-bottom: 20px;
            font-style: italic;
        }}

        .stats-box {{
            position: absolute;
            padding: 10px;
            border-radius: 4px;
            pointer-events: none;
        }}

        .region-stats {{
            background-color: white;
            color: black;
            border: 1px solid black;
            padding: 5px;
            margin-bottom: 5px;
        }}

        #region-stats-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            width: 180px; /* Increased width for more info */
            font-size: 0.9em;
        }}

    </style>
</head>
<body>
    <div class="container">
        <h1>HIV and AIDS Flow by UN Region ({start_year}-{end_year})</h1>

        <div class="controls">
            <button id="play-button">Play</button>
            <button id="pause-button">Pause</button>
        </div>

        <div class="year-display" id="year-display">Year: {start_year}</div>

        <div id="sankey-container">
            <div id="region-stats-container"></div>
        </div>

        <input type="range" id="year-slider" min="0" max="{len(available_years)-1}" value="0" step="1">

        <div class="explanation">
            This visualization shows the flow of HIV cases from a global total to UN regions, and then to AIDS-related deaths and non-AIDS related living with HIV over time.
        </div>
    </div>

    <script>
        const allYearsDataRegionsFlow = {json_data_regions_flow};
        const availableYears = Object.keys(allYearsDataRegionsFlow).sort((a, b) => parseInt(a) - parseInt(b));
        const unRegions = {json.dumps(un_regions)};

        let currentYearIndex = 0;
        let animationInterval;
        let isPlaying = false;

        // Get DOM elements
        const sankeyContainer = document.getElementById('sankey-container');
        const yearDisplay = document.getElementById('year-display');
        const yearSlider = document.getElementById('year-slider');
        const playButton = document.getElementById('play-button');
        const pauseButton = document.getElementById('pause-button');
        const regionStatsContainer = document.getElementById('region-stats-container');

        // Set up Sankey
        const margin = {{top: 10, right: 10, bottom: 10, left: 180}}; /* Increased left margin */
        const width = sankeyContainer.clientWidth - margin.left - margin.right;
        const height = sankeyContainer.clientHeight - margin.top - margin.bottom;

        const svg = d3.select('#sankey-container')
            .append('svg')
            .attr('width', width + margin.left + margin.right)
            .attr('height', height + margin.top + margin.bottom)
            .append('g')
            .attr('transform', `translate(${{margin.left}},${{margin.top}})`);

        const color = d3.scaleOrdinal(d3.schemeCategory10);

        updateSankey(allYearsDataRegionsFlow[availableYears[0]]);

        playButton.addEventListener('click', startAnimation);
        pauseButton.addEventListener('click', pauseAnimation);
        yearSlider.addEventListener('input', handleSliderChange);

        window.addEventListener('resize', () => {{
            updateSankey(allYearsDataRegionsFlow[availableYears[currentYearIndex]]);
        }});

        function updateSankey(yearData) {{
            svg.selectAll('*').remove();
            regionStatsContainer.innerHTML = '';
            yearDisplay.textContent = `Year: ${{availableYears[currentYearIndex]}}`;

            const nodes = [];
            const links = [];
            const regionNodeIndices = {{}};
            let nodeIndex = 0;

            // Initial node: Global Living with HIV
            nodes.push({{ name: 'People Living with HIV' }});
            const globalHIVIndex = nodeIndex++;

            // Region nodes
            for (const region of unRegions) {{
                nodes.push({{ name: region }});
                regionNodeIndices[region] = nodeIndex++;
            }}

            // Final nodes
            nodes.push({{ name: 'Living with HIV (non-AIDS)' }});
            const nonAidsIndex = nodeIndex++;
            nodes.push({{ name: 'AIDS-related Deaths' }});
            const aidsDeathIndex = nodeIndex++;

            // Links from Global HIV to Regions
            const globalHIVValue = yearData.global_hiv;
            if (globalHIVValue > 0) {{
                const totalRegionalHIV = Object.values(yearData.regions).reduce((sum, r) => sum + r.hiv, 0);
                for (const region of unRegions) {{
                    const regionData = yearData.regions[region];
                    if (regionData && regionData.hiv > 0) {{
                        links.push({{
                            source: globalHIVIndex,
                            target: regionNodeIndices[region],
                            value: regionData.hiv * (globalHIVValue > 0 ? 1 : 0), // Avoid division by zero
                            color: 'rgba(100, 149, 237, 0.6)' // Cornflower blue
                        }});
                        // Add region stats
                        const regionDiv = document.createElement('div');
                        regionDiv.className = 'region-stats';
                        regionDiv.innerHTML = `<strong>${{region}}</strong><br>HIV: ${{formatNumber(regionData.hiv)}}<br>Deaths: ${{formatNumber(regionData.aids_deaths)}}<br>Living (non-AIDS): ${{formatNumber(regionData.non_aids)}}`;
                        regionStatsContainer.appendChild(regionDiv);
                    }}
                }}
            }}

            // Links from Regions to Outcomes
            for (const region of unRegions) {{
                const regionData = yearData.regions[region];
                if (regionData) {{
                    const regionSource = regionNodeIndices[region];
                    if (regionData.non_aids > 0) {{
                        links.push({{
                            source: regionSource,
                            target: nonAidsIndex,
                            value: regionData.non_aids,
                            color: 'rgba(0, 255, 0, 0.6)' // Green
                        }});
                    }}
                    if (regionData.aids_deaths > 0) {{
                        links.push({{
                            source: regionSource,
                            target: aidsDeathIndex,
                            value: regionData.aids_deaths,
                            color: 'rgba(255, 0, 0, 0.6)' // Red
                        }});
                    }}
                }}
            }}

            const sankey = d3.sankey()
                .nodeAlign(d3.sankeyLeft)
                .nodeWidth(15)
                .nodePadding(10)
                .extent([[0, 0], [width, height]]);

            const {{nodes: graphNodes, links: graphLinks}} = sankey({{ nodes: nodes, links: links }});

            const node = svg.append('g')
                .attr('class', 'nodes')
                .selectAll('g')
                .data(graphNodes)
                .join('g');

            node.append('rect')
                .attr('x', d => d.x0)
                .attr('y', d => d.y0)
                .attr('height', d => d.y1 - d.y0)
                .attr('width', d => d.x1 - d.x0)
                .attr('fill', d => color(d.name))
                .style('stroke', '#000')
                .style('stroke-width', '0.5px')
                .append('title')
                .text(d => `${{d.name}}\\n${{formatNumber(d.value)}}`);

            node.append('text')
                .attr('x', d => d.x0 - 5)
                .attr('y', d => (d.y1 + d.y0) / 2)
                .attr('dy', '0.35em')
                .attr('text-anchor', 'end')
                .style('font-size', '10px')
                .text(d => d.name)
                .filter(d => d.x0 < width / 2);

            node.filter(d => d.x0 < width / 2)
                .select('text')
                .attr('x', d => d.x1 + 5)
                .attr('text-anchor', 'start');

            const link = svg.append('g')
                .attr('class', 'links')
                .selectAll('path')
                .data(graphLinks)
                .join('path')
                .attr('d', d3.sankeyLinkHorizontal())
                .attr('stroke', d => d.color)
                .style('stroke-opacity', 0.6)
                .style('fill', 'none')
                .attr('stroke-width', d => Math.max(1, d.width))
                .sort((a, b) => b.width - a.width);

            link.append('title')
                .text(d => `${{d.source.name}} → ${{d.target.name}}\\n${{formatNumber(d.value)}}`);
        }}

        function startAnimation() {{
            if (isPlaying) return;
            isPlaying = true;
            animationInterval = setInterval(() => {{
                currentYearIndex = (currentYearIndex + 1) % availableYears.length;
                yearSlider.value = currentYearIndex;
                updateSankey(allYearsDataRegionsFlow[availableYears[currentYearIndex]]);
                if (currentYearIndex === availableYears.length - 1) {{
                    pauseAnimation();
                }}
            }}, 500);
        }}

        function pauseAnimation() {{
            clearInterval(animationInterval);
            isPlaying = false;
        }}

        function handleSliderChange() {{
            pauseAnimation();
            currentYearIndex = parseInt(yearSlider.value);
            updateSankey(allYearsDataRegionsFlow[availableYears[currentYearIndex]]);
        }}

        function formatNumber(num) {{
            return num.toString().replace(/\\B(?=(\\d{3})+(?!\\d))/g, ",");
        }}
    </script>
</body>
</html>
"""

os.makedirs('../../interactive_viz_outputs', exist_ok=True)
output_path = '../../interactive_viz_outputs/hiv_aids_flow_by_region_multi_stage.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Multi-stage regional flow visualization created and saved as '{output_path}'")

Multi-stage regional flow visualization created and saved as '../../interactive_viz_outputs/hiv_aids_flow_by_region_multi_stage.html'


In [16]:
%%javascript
tooltip.transition()
    .duration(200)
    .style('opacity', 0.9);
    
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HIV and AIDS Flow Diagram</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://unpkg.com/d3-sankey@0.12.3/dist/d3-sankey.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #121212;
            color: white;
        }
        
        .container {
            max-width: 1200px;
            margin: 0 auto;
        }
        
        h1 {
            text-align: center;
            margin-bottom: 20px;
        }
        
        .controls {
            display: flex;
            justify-content: space-between;
            margin-bottom: 20px;
            gap: 10px;
            flex-wrap: wrap;
        }
        
        .playback-controls {
            display: flex;
            gap: 10px;
        }
        
        .view-controls {
            display: flex;
            gap: 10px;
        }
        
        button {
            padding: 10px 20px;
            background-color: #333;
            color: white;
            border: 1px solid #666;
            border-radius: 4px;
            cursor: pointer;
        }
        
        button:hover {
            background-color: #444;
        }
        
        button.active {
            background-color: #e53935;
            border-color: #e53935;
        }
        
        .year-display {
            text-align: center;
            font-size: 24px;
            margin-bottom: 20px;
        }
        
        #sankey-container {
            width: 100%;
            height: 500px; 
            position: relative;
            margin: 0 auto;
        }
        
        #year-slider {
            width: 100%;
            margin: 20px 0;
        }
        
        .node rect {
            cursor: move;
            fill-opacity: 0.9;
            shape-rendering: crispEdges;
        }
        
        .node text {
            pointer-events: none;
            text-shadow: 0 1px 0 #000;
            font-size: 12px;
            fill: white;
        }
        
        .link {
            fill: none;
            stroke-opacity: 0.5;
        }
        
        .tooltip {
            position: absolute;
            background-color: white;
            color: #121212;
            padding: 10px;
            border-radius: 5px;
            pointer-events: none;
            box-shadow: 0 0 10px rgba(0,0,0,0.5);
            z-index: 100;
        }
        
        .explanation {
            text-align: center;
            margin-top: 30px;
            margin-bottom: 20px;
            font-style: italic;
        }
        
        /* STYLING FOR STATS BOXES --> MODIFY POSITIONS HERE */
        .stats-box {
            position: absolute;
            padding: 10px;
            border-radius: 4px;
            pointer-events: none;
        }
        
        .hiv-stats {
            top: 35%;  /* ADJUST TO MOVE box up/down */
            right: 10px;  /* ADJUST TO MOVE box left/right */
            background-color: white;
            color: #fc9097;
            border: 2px solid #fc9097;
        }
        
        .aids-stats {
            bottom: 10%;  /* ADJUST TO MOVE box up/down */
            right: 10px;  /* ADJUST TO MOVE box left/right */
            background-color: white;
            color: red;
            border: 2px solid red;
        }
        
        /* STYLING FOR STARTING POINT BOX */
        .starting-note {
            position: absolute;
            top: 25%;  /* ADJUST TO MOVE box up/down */
            left: 80px;  /* ADJUST TO MOVE box left/right */
            background-color: #121212;
            color: #fc656f;  
            border: 2px solid #fc656f;
            padding: 10px;
            border-radius: 4px;
            pointer-events: none;
        }
        
        .legend {
            margin-top: 20px;
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            gap: 15px;
        }
        
        .legend-item {
            display: flex;
            align-items: center;
            margin-right: 15px;
        }
        
        .legend-color {
            width: 15px;
            height: 15px;
            margin-right: 5px;
            border-radius: 3px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>HIV and AIDS Flow Diagram (1990-2022)</h1>
        
        <div class="controls">
            <div class="playback-controls">
                <button id="play-button">Play</button>
                <button id="pause-button">Pause</button>
            </div>
            <div class="view-controls">
                <button id="basic-view" class="active">Basic View</button>
                <button id="region-view">Regional Breakdown</button>
            </div>
        </div>
        
        <div class="year-display" id="year-display">Year: 1990</div>
        
        <div id="sankey-container">
            <div class="starting-note" id="starting-note">All 8,910,444 HIV cases in 1990</div>
            <div class="stats-box hiv-stats" id="hiv-stats"></div>
            <div class="stats-box aids-stats" id="aids-stats"></div>
        </div>
        
        <input type="range" id="year-slider" min="0" max="32" value="0" step="1">
        
        <div id="legend" class="legend"></div>
        
        <div class="explanation">
            This visualization shows how the proportion of AIDS-related deaths has decreased over time.<br>
            Modern treatments have allowed more people to live with HIV without developing AIDS.
        </div>
    </div>
    
    <script>
        // Example data (simplified - you would replace this with your actual data)
        const allYearsData = [
            {
                year: 1990,
                total_hiv: 8910444,
                aids_deaths: 342343,
                non_aids: 8568101,
                death_percentage: 3.8,
                survival_percentage: 96.2
            },
            {
                year: 1991,
                total_hiv: 9876543,
                aids_deaths: 356789,
                non_aids: 9519754,
                death_percentage: 3.6,
                survival_percentage: 96.4
            },
            // Additional years would be included here
            {
                year: 2022,
                total_hiv: 39000000,
                aids_deaths: 630000,
                non_aids: 38370000,
                death_percentage: 1.6,
                survival_percentage: 98.4
            }
        ];
        
        // Region data (simplified - you would replace this with your actual data)
        const regionNames = {
            "UNAAP": "Asia and Pacific",
            "UNACAR": "Caribbean",
            "UNAESA": "Eastern and Southern Africa",
            "UNAEECA": "Eastern Europe and Central Asia",
            "UNALA": "Latin America",
            "UNAMENA": "Middle East and North Africa",
            "UNAWCA": "Western and Central Africa",
            "UNAWCENA": "Western and Central Europe and North America"
        };
        
        const regionColors = {
            "UNAAP": "#1f77b4",    // blue
            "UNACAR": "#ff7f0e",   // orange
            "UNAESA": "#2ca02c",   // green
            "UNAEECA": "#d62728",  // red
            "UNALA": "#9467bd",    // purple
            "UNAMENA": "#8c564b",  // brown
            "UNAWCA": "#e377c2",   // pink
            "UNAWCENA": "#7f7f7f"  // gray
        };
        
        const allYearsRegionsData = [
            {
                year: 1990,
                total_hiv: 8910444,
                aids_deaths: 342343,
                non_aids: 8568101,
                death_percentage: 3.8,
                survival_percentage: 96.2,
                regions: [
                    {
                        region_code: "UNAESA",
                        region_name: "Eastern and Southern Africa",
                        region_color: "#2ca02c",
                        hiv: 5000000,
                        aids_deaths: 200000,
                        non_aids: 4800000,
                        death_percentage: 4.0,
                        survival_percentage: 96.0
                    },
                    {
                        region_code: "UNAWCA",
                        region_name: "Western and Central Africa",
                        region_color: "#e377c2",
                        hiv: 1500000,
                        aids_deaths: 60000,
                        non_aids: 1440000,
                        death_percentage: 4.0,
                        survival_percentage: 96.0
                    },
                    {
                        region_code: "UNAAP",
                        region_name: "Asia and Pacific",
                        region_color: "#1f77b4",
                        hiv: 1000000,
                        aids_deaths: 40000,
                        non_aids: 960000,
                        death_percentage: 4.0,
                        survival_percentage: 96.0
                    },
                    {
                        region_code: "UNAWCENA",
                        region_name: "Western and Central Europe and North America",
                        region_color: "#7f7f7f",
                        hiv: 1000000,
                        aids_deaths: 30000,
                        non_aids: 970000,
                        death_percentage: 3.0,
                        survival_percentage: 97.0
                    },
                    {
                        region_code: "UNALA",
                        region_name: "Latin America",
                        region_color: "#9467bd",
                        hiv: 300000,
                        aids_deaths: 10000,
                        non_aids: 290000,
                        death_percentage: 3.3,
                        survival_percentage: 96.7
                    },
                    {
                        region_code: "UNACAR",
                        region_name: "Caribbean",
                        region_color: "#ff7f0e",
                        hiv: 100000,
                        aids_deaths: 2000,
                        non_aids: 98000,
                        death_percentage: 2.0,
                        survival_percentage: 98.0
                    },
                    {
                        region_code: "UNAEECA",
                        region_name: "Eastern Europe and Central Asia",
                        region_color: "#d62728",
                        hiv: 10000,
                        aids_deaths: 300,
                        non_aids: 9700,
                        death_percentage: 3.0,
                        survival_percentage: 97.0
                    },
                    {
                        region_code: "UNAMENA",
                        region_name: "Middle East and North Africa",
                        region_color: "#8c564b",
                        hiv: 444,
                        aids_deaths: 43,
                        non_aids: 401,
                        death_percentage: 9.7,
                        survival_percentage: 90.3
                    }
                ]
            },
            // Additional years would be included here
            {
                year: 2022,
                total_hiv: 39000000,
                aids_deaths: 630000,
                non_aids: 38370000,
                death_percentage: 1.6,
                survival_percentage: 98.4,
                regions: [
                    {
                        region_code: "UNAESA",
                        region_name: "Eastern and Southern Africa",
                        region_color: "#2ca02c",
                        hiv: 20500000,
                        aids_deaths: 350000,
                        non_aids: 20150000,
                        death_percentage: 1.7,
                        survival_percentage: 98.3
                    },
                    {
                        region_code: "UNAWCA",
                        region_name: "Western and Central Africa",
                        region_color: "#e377c2",
                        hiv: 5000000,
                        aids_deaths: 120000,
                        non_aids: 4880000,
                        death_percentage: 2.4,
                        survival_percentage: 97.6
                    },
                    {
                        region_code: "UNAAP",
                        region_name: "Asia and Pacific",
                        region_color: "#1f77b4",
                        hiv: 6000000,
                        aids_deaths: 80000,
                        non_aids: 5920000,
                        death_percentage: 1.3,
                        survival_percentage: 98.7
                    },
                    {
                        region_code: "UNAWCENA",
                        region_name: "Western and Central Europe and North America",
                        region_color: "#7f7f7f",
                        hiv: 2300000,
                        aids_deaths: 13000,
                        non_aids: 2287000,
                        death_percentage: 0.6,
                        survival_percentage: 99.4
                    },
                    {
                        region_code: "UNALA",
                        region_name: "Latin America",
                        region_color: "#9467bd",
                        hiv: 2500000,
                        aids_deaths: 45000,
                        non_aids: 2455000,
                        death_percentage: 1.8,
                        survival_percentage: 98.2
                    },
                    {
                        region_code: "UNACAR",
                        region_name: "Caribbean",
                        region_color: "#ff7f0e",
                        hiv: 330000,
                        aids_deaths: 6000,
                        non_aids: 324000,
                        death_percentage: 1.8,
                        survival_percentage: 98.2
                    },
                    {
                        region_code: "UNAEECA",
                        region_name: "Eastern Europe and Central Asia",
                        region_color: "#d62728",
                        hiv: 1800000,
                        aids_deaths: 15000,
                        non_aids: 1785000,
                        death_percentage: 0.8,
                        survival_percentage: 99.2
                    },
                    {
                        region_code: "UNAMENA",
                        region_name: "Middle East and North Africa",
                        region_color: "#8c564b",
                        hiv: 570000,
                        aids_deaths: 7000,
                        non_aids: 563000,
                        death_percentage: 1.2,
                        survival_percentage: 98.8
                    }
                ]
            }
        ];
        
        const availableYears = allYearsData.map(d => d.year);

        let currentYearIndex = 0;
        let animationInterval;
        let isPlaying = false;
        let showRegions = false;
        
        // Get DOM elements
        const sankeyContainer = document.getElementById('sankey-container');
        const yearDisplay = document.getElementById('year-display');
        const yearSlider = document.getElementById('year-slider');
        const playButton = document.getElementById('play-button');
        const pauseButton = document.getElementById('pause-button');
        const basicViewButton = document.getElementById('basic-view');
        const regionViewButton = document.getElementById('region-view');
        const hivStats = document.getElementById('hiv-stats');
        const aidsStats = document.getElementById('aids-stats');
        const startingNote = document.getElementById('starting-note');
        const legendContainer = document.getElementById('legend');
        
        // Set up sankey
        const margin = {top: 10, right: 100, bottom: 10, left: 100};
        const width = sankeyContainer.clientWidth - margin.left - margin.right;
        const height = sankeyContainer.clientHeight - margin.top - margin.bottom;
        
        const svg = d3.select('#sankey-container')
            .append('svg')
            .attr('width', width + margin.left + margin.right)
            .attr('height', height + margin.top + margin.bottom)
            .append('g')
            .attr('transform', `translate(${margin.left},${margin.top})`);
            
        // Add tooltip
        const tooltip = d3.select('#sankey-container')
            .append('div')
            .attr('class', 'tooltip')
            .style('opacity', 0);

        // Initialize with the first year's data
        updateSankey(allYearsData[0], allYearsRegionsData[0]);

        // Event listeners
        playButton.addEventListener('click', startAnimation);
        pauseButton.addEventListener('click', pauseAnimation);
        yearSlider.addEventListener('input', handleSliderChange);
        basicViewButton.addEventListener('click', () => toggleView(false));
        regionViewButton.addEventListener('click', () => toggleView(true));
        
        // Window resize handler
        window.addEventListener('resize', () => {
            // Redraw on window resize
            updateSankey(
                allYearsData[currentYearIndex], 
                allYearsRegionsData[currentYearIndex]
            );
        });
        
        // Functions
        function toggleView(showRegionalView) {
            showRegions = showRegionalView;
            
            // Update button states
            basicViewButton.classList.toggle('active', !showRegions);
            regionViewButton.classList.toggle('active', showRegions);
            
            // Update visualization
            updateSankey(
                allYearsData[currentYearIndex], 
                allYearsRegionsData[currentYearIndex]
            );
            
            // Show/hide starting note based on view
            startingNote.style.display = showRegions ? 'none' : 'block';
        }
        
        function updateSankey(yearData, yearRegionsData) {
            // Clear previous content
            svg.selectAll('*').remove();
            legendContainer.innerHTML = '';
            
            // Update year display
            yearDisplay.textContent = `Year: ${yearData.year}`;
            
            // Update stats
            hivStats.textContent = `Living with HIV: ${yearData.survival_percentage.toFixed(1)}%`;
            aidsStats.textContent = `AIDS-related Deaths: ${yearData.death_percentage.toFixed(1)}%`;

            if (!showRegions) {
                // Basic view without regions
                drawBasicSankey(yearData);
            } else {
                // Regional breakdown view
                drawRegionalSankey(yearRegionsData);
            }
        }
        
        function drawBasicSankey(yearData) {
            const totalFlow = yearData.aids_deaths + yearData.non_aids;
            const totalHeight = height * 0.8; // Use 80% of available height

            const hivNodeHeight = totalHeight;
            const nonAidsNodeHeight = (yearData.non_aids / totalFlow) * totalHeight;
            const aidsNodeHeight = (yearData.aids_deaths / totalFlow) * totalHeight;

            const nodes = [
                { 
                    id: 0,
                    name: 'HIV',
                    x0: 0,
                    x1: 20,
                    y0: (height - hivNodeHeight) / 2,
                    y1: (height - hivNodeHeight) / 2 + hivNodeHeight
                },
                { 
                    id: 1,
                    name: 'AIDS Deaths',
                    x0: width - 20,
                    x1: width,
                    y0: height - aidsNodeHeight,
                    y1: height
                },
                { 
                    id: 2,
                    name: 'Living with HIV',
                    x0: width - 20,
                    x1: width,
                    y0: 0,
                    y1: nonAidsNodeHeight
                }
            ];
            
            // Create links with proper values
            const links = [
                { 
                    source: nodes[0], 
                    target: nodes[1], 
                    value: yearData.aids_deaths,
                    width: aidsNodeHeight,
                    color: 'rgba(255, 0, 0, 0.6)'
                },
                { 
                    source: nodes[0], 
                    target: nodes[2], 
                    value: yearData.non_aids,
                    width: nonAidsNodeHeight,
                    color: 'rgba(252, 144, 151, 0.6)'
                }
            ];
            
            // Color scale
            const nodeColors = {'0': '#fc656f', '1': 'red', '2': '#fc9097'};
            
            // Draw nodes
            const nodes_g = svg.append('g')
                .attr('class', 'nodes')
                .selectAll('g')
                .data(nodes)
                .enter()
                .append('g');
            
            nodes_g.append('rect')
                .attr('x', d => d.x0)
                .attr('y', d => d.y0)
                .attr('height', d => d.y1 - d.y0)
                .attr('width', d => d.x1 - d.x0)
                .style('fill', d => nodeColors[d.id])
                .style('stroke', '#fff')
                .on('mouseover', function(event, d) {
                    tooltip.transition()
                        .duration(200)
                        .style('opacity', 0.9);
                    
                    let content = '';
                    if (d.id === 0) {
                        content = `Total HIV Cases: ${yearData.total_hiv.toLocaleString()}`;
                    } else if (d.id === 1) {
                        content = `AIDS Deaths: ${yearData.aids_deaths.toLocaleString()} (${yearData.death_percentage.toFixed(1)}%)`;
                    } else {
                        content = `Living with HIV: ${yearData.non_aids.toLocaleString()} (${yearData.survival_percentage.toFixed(1)}%)`;
                    }
                    
                    tooltip.html(content)
                        .style('left', (event.pageX + 10) + 'px')
                        .style('top', (event.pageY - 28) + 'px');
                })
                .on('mouseout', function() {
                    tooltip.transition()
                        .duration(500)
                        .style('opacity', 0);
                });
                
            // Add node labels
            nodes_g.append('text')
                .attr('x', d => d.id === 0 ? d.x1 + 5 : d.x0 - 5)
                .attr('y', d => (d.y0 + d.y1) / 2)
                .attr('dy', '0.35em')
                .attr('text-anchor', d => d.id === 0 ? 'start' : 'end')
                .text(d => d.name)
                .style('fill', 'white');

            // Draw links
            svg.append('g')
                .attr('class', 'links')
                .selectAll('path')
                .data(links)
                .enter()
                .append('path')
                .attr('d', d => {
                    const sourceX = d.source.x1;
                    const sourceY = d.source.y0 + (d.source.y1 - d.source.y0) / 2;
                    const targetX = d.target.x0;
                    const targetY = d.target.y0 + (d.target.y1 - d.target.y0) / 2;

                    const controlX1 = sourceX + (targetX - sourceX) / 3;
                    const controlX2 = sourceX + 2 * (targetX - sourceX) / 3;
                    
                    // Adjust for flow target (AIDS deaths = bottom, HIV = top)
                    const controlY1 = sourceY;
                    const controlY2 = targetY;

                    return `M${sourceX},${sourceY} C${controlX1},${controlY1} ${controlX2},${controlY2} ${targetX},${targetY}`;
                })
                .attr('stroke', d => d.color)
                .attr('stroke-width', d => Math.max(1, d.width))
                .style('fill', 'none')
                .on('mouseover', function(event, d) {
                    tooltip.transition()
                        .duration(200)
                        .style('opacity', 0.9);
                    
                    let content = '';
                    if (d.target.id === 1) {
                        content = `AIDS Deaths: ${d.value.toLocaleString()} (${yearData.death_percentage.toFixed(1)}%)`;
                    } else {
                        content = `Living with HIV: ${d.value.toLocaleString()} (${yearData.survival_percentage.toFixed(1)}%)`;
                    }
                    
                    tooltip.html(content)
                        .style('left', (event.pageX + 10) + 'px')
                        .style('top', (event.pageY - 28) + 'px');
                })
                .on('mouseout', function() {
                    tooltip.transition()
                        .duration(500)
                        .style('opacity', 0);
                });
        }
        
        function drawRegionalSankey(yearRegionsData) {
            const regions = yearRegionsData.regions;
            
            // Skip if no regions data
            if (!regions || regions.length === 0) {
                svg.append('text')
                    .attr('x', width / 2)
                    .attr('y', height / 2)
                    .attr('text-anchor', 'middle')
                    .text('No regional data available for this year')
                    .style('fill', 'white');
                return;
            }
            
            // Create nodes array
            let nodes = [];
            
            // HIV central node
            nodes.push({
                id: 0,
                name: 'HIV',
                type: 'central',
                x0: 0,
                x1: 20,
                y0: height * 0.1,
                y1: height * 0.9
            });
            
            // Region nodes (index 1 to regions.length)
            const regionHeight = (height * 0.8) / regions.length;
            regions.forEach((region, idx) => {
                nodes.push({
                    id: idx + 1,
                    name: region.region_name,
                    code: region.region_code,
                    type: 'region',
                    color: region.region_color,
                    hiv: region.hiv,
                    x0: width * 0.4 - 20,
                    x1: width * 0.4,
                    y0: height * 0.1 + idx * regionHeight,
                    y1: height * 0.1 + (idx + 1) * regionHeight
                });
            });
            
            // Outcome nodes (AIDS Deaths and Living with HIV)
            nodes.push({
                id: nodes.length,
                name: 'AIDS Deaths',
                type: 'outcome',
                x0: width - 20,
                x1: width,
                y0: height * 0.7,
                y1: height * 0.9
            });
            
            nodes.push({
                id: nodes.length,
                name: 'Living with HIV',
                type: 'outcome',
                x0: width - 20,
                x1: width,
                y0: height * 0.1,
                y1: height * 0.3
            });
            
            // Create links
            let links = [];
            
            // Links from HIV to Regions
            regions.forEach((region, idx) => {
                links.push({
                    source: nodes[0],
                    target: nodes[idx + 1],
                    value: region.hiv,
                    color: region.region_color
                });
            });
            
            // Links from Regions to Outcomes
            const aidsNodeIndex = nodes.length - 2;  // Second to last node
            const hivNodeIndex = nodes.length - 1;   // Last node
            
            regions.forEach((region, idx) => {
                // Link to AIDS Deaths
                links.push({
                    source: nodes[idx + 1],
                    target: nodes[aidsNodeIndex],
                    value: region.aids_deaths,
                    color: 'rgba(255, 0, 0, 0.6)'
                });
                
                // Link to Living with HIV
                links.push({
                    source: nodes[idx + 1],
                    target: nodes[hivNodeIndex],
                    value: region.non_aids,
                    color: 'rgba(252, 144, 151, 0.6)'
                });
            });
            
            // Draw nodes
            const nodes_g = svg.append('g')
                .attr('class', 'nodes')
                .selectAll('g')
                .data(nodes)
                .enter()
                .append('g');
            
            nodes_g.append('rect')
                .attr('x', d => d.x0)
                .attr('y', d => d.y0)
                .attr('height', d => d.y1 - d.y0)
                .attr('width', d => d.x1 - d.x0)
                .style('fill', d => {
                    if (d.type === 'central') return '#fc656f';
                    if (d.type === 'region') return d.color;
                    if (d.name === 'AIDS Deaths') return 'red';
                    return '#fc9097';
                })
                .style('stroke', '#fff')
                .on('mouseover', function(event, d) {
                    tooltip.transition()
                    .duration(200)
                    .style('opacity', 0.9);
                    
                    let content = '';
                    if (d.type === 'central') {
                    content = `Total HIV Cases: ${yearRegionsData.total_hiv.toLocaleString()}`;
                    } else if (d.type === 'region') {
                    content = `${d.name}: ${d.hiv.toLocaleString()} cases`;
                    } else if (d.name === 'AIDS Deaths') {
                    content = `AIDS Deaths: ${yearRegionsData.aids_deaths.toLocaleString()} (${yearRegionsData.death_percentage.toFixed(1)}%)`;
                    } else {
                    content = `Living with HIV: ${yearRegionsData.non_aids.toLocaleString()} (${yearRegionsData.survival_percentage.toFixed(1)}%)`;
                    }
                    
                    tooltip.html(content)
                    .style('left', (event.pageX + 10) + 'px')
                    .style('top', (event.pageY - 28) + 'px');
                })
                .on('mouseout', function() {
                    tooltip.transition()
                    .duration(500)
                    .style('opacity', 0);
                });

                    
            // Add node labels
            nodes_g.append('text')
                .attr('x', d => {
                    if (d.type === 'central') return d.x1 + 5;
                    if (d.type === 'region') return d.x0 - 5;
                    return d.x0 - 5;
                })
                .attr('y', d => (d.y0 + d.y1) / 2)
                .attr('dy', '0.35em')
                .attr('text-anchor', d => {
                    if (d.type === 'central') return 'start';
                    return 'end';
                })
                .text(d => {
                    if (d.type === 'region') {
                        return d.name;
                    }
                    return d.name;
                })
                .style('fill', 'white');

            // Draw links
            svg.append('g')
                .attr('class', 'links')
                .selectAll('path')
                .data(links)
                .enter()
                .append('path')
                .attr('d', d => {
                    const sourceX = d.source.x1;
                    const sourceY = d.source.y0 + (d.source.y1 - d.source.y0) / 2;
                    const targetX = d.target.x0;
                    const targetY = d.target.y0 + (d.target.y1 - d.target.y0) / 2;

                    const controlX1 = sourceX + (targetX - sourceX) / 3;
                    const controlX2 = sourceX + 2 * (targetX - sourceX) / 3;
                    
                    const controlY1 = sourceY;
                    const controlY2 = targetY;

                    return `M${sourceX},${sourceY} C${controlX1},${controlY1} ${controlX2},${controlY2} ${targetX},${targetY}`;
                })
                .attr('stroke', d => d.color)
                .attr('stroke-width', d => Math.max(1, Math.sqrt(d.value) / 10))
                .style('fill', 'none')
                .on('mouseover', function(event, d) {
                    tooltip.transition()
                        .duration(200)
                        .style('opacity', 0.9);
                    
                    let content = '';
                    const regionIndex = d.source.id - 1;
                    
                    if (d.source.type === 'central') {
                        const region = regions[d.target.id - 1];
                        content = `HIV Cases in ${d.target.name}: ${d.value.toLocaleString()}`;
                    } else if (d.target.name === 'AIDS Deaths') {
                        const region = regions[regionIndex];
                        content = `AIDS Deaths in ${d.source.name}: ${d.value.toLocaleString()} (${region.death_percentage.toFixed(1)}%)`;
                    } else {
                        const region = regions[regionIndex];
                        content = `Living with HIV in ${d.source.name}: ${d.value.toLocaleString()} (${region.survival_percentage.toFixed(1)}%)`;
                    }
                    
                    tooltip.html(content)
                        .style('left', (event.pageX + 10) + 'px')
                        .style('top', (event.pageY - 28) + 'px');
                })
                .on('mouseout', function() {
                    tooltip.transition()
                        .duration(500)
                        .style('opacity', 0);
                });
                
            // Create legend
            const legendItems = regions.map(region => ({
                name: region.region_name,
                color: region.region_color
            }));
            
            const legend = d3.select('#legend');
            
            legendItems.forEach(item => {
                const legendItem = document.createElement('div');
                legendItem.className = 'legend-item';
                
                const colorBox = document.createElement('div');
                colorBox.className = 'legend-color';
                colorBox.style.backgroundColor = item.color;
                
                const legendText = document.createElement('span');
                legendText.textContent = item.name;
                
                legendItem.appendChild(colorBox);
                legendItem.appendChild(legendText);
                legendContainer.appendChild(legendItem);
            });
        }
        
        function startAnimation() {
            if (isPlaying) return;
            
            isPlaying = true;
            animationInterval = setInterval(() => {
                currentYearIndex = (currentYearIndex + 1) % availableYears.length;
                yearSlider.value = currentYearIndex;
                updateSankey(
                    allYearsData[currentYearIndex],
                    allYearsRegionsData[currentYearIndex]
                );
                
                if (currentYearIndex === availableYears.length - 1) {
                    // Reached the end
                    pauseAnimation();
                }
            }, 500); // 500ms per frame
        }
        
        function pauseAnimation() {
            clearInterval(animationInterval);
            isPlaying = false;
        }
        
        function handleSliderChange() {
            pauseAnimation();
            
            // Update year
            currentYearIndex = parseInt(yearSlider.value);
            updateSankey(
                allYearsData[currentYearIndex],
                allYearsRegionsData[currentYearIndex]
            );
        }
    </script>
</body>
</html>

import os

# Create directory if it doesn't exist
os.makedirs('../../interactive_viz_outputs', exist_ok=True)

# Export the HTML file
output_path = '../../interactive_viz_outputs/hiv_aids_flow_diagram_with_regions.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Visualization created and saved as '{output_path}'")

<IPython.core.display.Javascript object>

In [17]:
import pandas as pd
import json
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the cleaned data
df = pd.read_excel('../../data/clean_data/cleaned_UNAIDS.xlsx')

# Define UN regions
un_regions = ["UNAAP", "UNACAR", "UNAESA", "UNAEECA", "UNALA", "UNAMENA", "UNAWCA", "UNAWCENA"]

# Get available years
available_years = sorted(df['Year'].unique())
start_year = min(available_years)
end_year = max(available_years)

all_years_data_regions = {}

for year in available_years:
    year_data = df[df['Year'] == year].copy()
    year_regions_data = {}
    for region in un_regions:
        region_data = year_data[year_data['Country Code'] == region].copy()
        region_data['AIDS-related deaths among adults and children'] = region_data['AIDS-related deaths among adults and children'].fillna(0)
        region_data['Estimated adults and children living with HIV'] = region_data['Estimated adults and children living with HIV'].fillna(0)
        region_data['Living with HIV (non-AIDS deaths)'] = region_data['Estimated adults and children living with HIV'] - region_data['AIDS-related deaths among adults and children']
        region_data['Living with HIV (non-AIDS deaths)'] = region_data['Living with HIV (non-AIDS deaths)'].clip(lower=0)

        total_hiv = region_data['Estimated adults and children living with HIV'].sum()
        total_aids_deaths = region_data['AIDS-related deaths among adults and children'].sum()
        total_non_aids = total_hiv - total_aids_deaths

        year_regions_data[region] = {
            'total_hiv': float(total_hiv),
            'aids_deaths': float(total_aids_deaths),
            'non_aids': float(total_non_aids),
        }
    all_years_data_regions[int(year)] = year_regions_data

# Convert data to JSON
json_data_regions = json.dumps(all_years_data_regions)

# HTML file with embedded data and D3.js
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HIV and AIDS Flow by UN Region ({start_year}-{end_year})</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://unpkg.com/d3-sankey@0.12.3/dist/d3-sankey.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #121212;
            color: white;
        }}

        .container {{
            max-width: 1200px;
            margin: 0 auto;
        }}

        h1 {{
            text-align: center;
            margin-bottom: 20px;
        }}

        .controls {{
            display: flex;
            justify-content: flex-start;
            margin-bottom: 20px;
            gap: 10px;
        }}

        button {{
            padding: 10px 20px;
            background-color: #333;
            color: white;
            border: 1px solid #666;
            border-radius: 4px;
            cursor: pointer;
        }}

        button:hover {{
            background-color: #444;
        }}

        .year-display {{
            text-align: center;
            font-size: 24px;
            margin-bottom: 20px;
        }}

        #sankey-container {{
            width: 100%;
            height: 600px;
            position: relative;
            margin: 0 auto;
        }}

        #year-slider {{
            width: 100%;
            margin: 20px 0;
        }}

        .node rect {{
            cursor: move;
            fill-opacity: 0.9;
            shape-rendering: crispEdges;
        }}

        .node text {{
            pointer-events: none;
            text-shadow: 0 1px 0 #000;
        }}

        .link {{
            fill: none;
            stroke-opacity: 0.5;
        }}

        .tooltip {{
            position: absolute;
            background-color: white;
            color: #121212;
            padding: 10px;
            border-radius: 5px;
            pointer-events: none;
            box-shadow: 0 0 10px rgba(0,0,0,0.5);
        }}

        .explanation {{
            text-align: center;
            margin-top: 30px;
            margin-bottom: 20px;
            font-style: italic;
        }}

        .stats-box {{
            position: absolute;
            padding: 10px;
            border-radius: 4px;
            pointer-events: none;
        }}

        .region-stats {{
            background-color: white;
            color: black;
            border: 1px solid black;
            padding: 5px;
            margin-bottom: 5px;
        }}

        /* Adjust positioning of stats boxes as needed */
        #region-stats-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            width: 150px;
        }}

    </style>
</head>
<body>
    <div class="container">
        <h1>HIV and AIDS Flow by UN Region ({start_year}-{end_year})</h1>

        <div class="controls">
            <button id="play-button">Play</button>
            <button id="pause-button">Pause</button>
        </div>

        <div class="year-display" id="year-display">Year: {start_year}</div>

        <div id="sankey-container">
            <div id="region-stats-container"></div>
        </div>

        <input type="range" id="year-slider" min="0" max="{len(available_years)-1}" value="0" step="1">

        <div class="explanation">
            This visualization shows the flow of HIV cases to AIDS-related deaths and non-AIDS related living with HIV, broken down by UN region over time.
        </div>
    </div>

    <script>
        const allYearsDataRegions = {json_data_regions};
        const availableYears = Object.keys(allYearsDataRegions).sort((a, b) => parseInt(a) - parseInt(b));
        const unRegions = {json.dumps(un_regions)};

        let currentYearIndex = 0;
        let animationInterval;
        let isPlaying = false;

        // Get DOM elements
        const sankeyContainer = document.getElementById('sankey-container');
        const yearDisplay = document.getElementById('year-display');
        const yearSlider = document.getElementById('year-slider');
        const playButton = document.getElementById('play-button');
        const pauseButton = document.getElementById('pause-button');
        const regionStatsContainer = document.getElementById('region-stats-container');

        // Set up Sankey
        const margin = {{top: 10, right: 10, bottom: 10, left: 150}};
        const width = sankeyContainer.clientWidth - margin.left - margin.right;
        const height = sankeyContainer.clientHeight - margin.top - margin.bottom;

        const svg = d3.select('#sankey-container')
            .append('svg')
            .attr('width', width + margin.left + margin.right)
            .attr('height', height + margin.top + margin.bottom)
            .append('g')
            .attr('transform', `translate(${{margin.left}},${{margin.top}})`);

        const color = d3.scaleOrdinal(d3.schemeCategory10);

        updateSankey(allYearsDataRegions[availableYears[0]]);

        playButton.addEventListener('click', startAnimation);
        pauseButton.addEventListener('click', pauseAnimation);
        yearSlider.addEventListener('input', handleSliderChange);

        window.addEventListener('resize', () => {{
            updateSankey(allYearsDataRegions[availableYears[currentYearIndex]]);
        }});

        function updateSankey(yearData) {{
            svg.selectAll('*').remove();
            regionStatsContainer.innerHTML = '';
            yearDisplay.textContent = `Year: ${{availableYears[currentYearIndex]}}`;

            const nodes = [];
            const links = [];
            const regionNodeIndices = {{}};
            let nodeIndex = 0;

            // Create initial nodes (Living with HIV)
            for (const region of unRegions) {{
                nodes.push({{ name: `Living with HIV (${{region}})` }});
                regionNodeIndices[`hiv-${{region}}`] = nodeIndex++;
            }}
            nodes.push({{ name: 'AIDS-related Deaths' }});
            const aidsDeathIndex = nodeIndex++;
            nodes.push({{ name: 'Living with HIV (non-AIDS)' }});
            const nonAidsIndex = nodeIndex++;

            // Create links
            for (const region of unRegions) {{
                const data = yearData[region];
                if (data) {{
                    const hivSource = regionNodeIndices[`hiv-${{region}}`];
                    if (data.aids_deaths > 0) {{
                        links.push({{ source: hivSource, target: aidsDeathIndex, value: data.aids_deaths, color: 'rgba(255, 0, 0, 0.6)' }});
                    }}
                    if (data.non_aids > 0) {{
                        links.push({{ source: hivSource, target: nonAidsIndex, value: data.non_aids, color: 'rgba(0, 255, 0, 0.6)' }});
                    }}

                    // Add stats for each region
                    const regionDiv = document.createElement('div');
                    regionDiv.className = 'region-stats';
                    regionDiv.innerHTML = `<strong>${{region}}</strong><br>HIV: ${{formatNumber(data.total_hiv)}}<br>Deaths: ${{formatNumber(data.aids_deaths)}}<br>Living (non-AIDS): ${{formatNumber(data.non_aids)}}`;
                    regionStatsContainer.appendChild(regionDiv);
                }}
            }}

            const sankey = d3.sankey()
                .nodeAlign(d3.sankeyLeft)
                .nodeWidth(15)
                .nodePadding(10)
                .extent([[0, 0], [width, height]]);

            const {{nodes: graphNodes, links: graphLinks}} = sankey({{ nodes: nodes, links: links }});

            const node = svg.append('g')
                .attr('class', 'nodes')
                .selectAll('g')
                .data(graphNodes)
                .join('g');

            node.append('rect')
                .attr('x', d => d.x0)
                .attr('y', d => d.y0)
                .attr('height', d => d.y1 - d.y0)
                .attr('width', d => d.x1 - d.x0)
                .attr('fill', d => color(d.name.replace(/ \\(.*\\)/, '')))
                .style('stroke', '#000')
                .style('stroke-width', '0.5px')
                .append('title')
                .text(d => `${{d.name}}\\n${{formatNumber(d.value)}}`);

            node.append('text')
                .attr('x', d => d.x0 - 5)
                .attr('y', d => (d.y1 + d.y0) / 2)
                .attr('dy', '0.35em')
                .attr('text-anchor', 'end')
                .style('font-size', '10px')
                .text(d => d.name)
                .filter(d => d.x0 < width / 2);

            node.filter(d => d.x0 < width / 2)
                .select('text')
                .attr('x', d => d.x1 + 5)
                .attr('text-anchor', 'start');

            const link = svg.append('g')
                .attr('class', 'links')
                .selectAll('path')
                .data(graphLinks)
                .join('path')
                .attr('d', d3.sankeyLinkHorizontal())
                .attr('stroke', d => d.color)
                .style('stroke-opacity', 0.6)
                .style('fill', 'none')
                .attr('stroke-width', d => Math.max(1, d.width))
                .sort((a, b) => b.width - a.width);

            link.append('title')
                .text(d => `${{d.source.name}} → ${{d.target.name}}\\n${{formatNumber(d.value)}}`);
        }}

        function startAnimation() {{
            if (isPlaying) return;
            isPlaying = true;
            animationInterval = setInterval(() => {{
                currentYearIndex = (currentYearIndex + 1) % availableYears.length;
                yearSlider.value = currentYearIndex;
                updateSankey(allYearsDataRegions[availableYears[currentYearIndex]]);
                if (currentYearIndex === availableYears.length - 1) {{
                    pauseAnimation();
                }}
            }}, 500);
        }}

        function pauseAnimation() {{
            clearInterval(animationInterval);
            isPlaying = false;
        }}

        function handleSliderChange() {{
            pauseAnimation();
            currentYearIndex = parseInt(yearSlider.value);
            updateSankey(allYearsDataRegions[availableYears[currentYearIndex]]);
        }}

        function formatNumber(num) {{
            return num.toString().replace(/\\B(?=(\\d{3})+(?!\\d))/g, ",");
        }}
    </script>
</body>
</html>
"""

os.makedirs('../../interactive_viz_outputs', exist_ok=True)
output_path = '../../interactive_viz_outputs/hiv_aids_flow_by_region_styled.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Visualization created and saved as '{output_path}'")

Visualization created and saved as '../../interactive_viz_outputs/hiv_aids_flow_by_region_styled.html'
